In [1]:
import numpy as np
import cv2

In [2]:
model_cfg = "model/deploy.prototxt.txt"
model_weights = "model/res10_300x300_ssd_iter_140000.caffemodel"

# read the model
detector = cv2.dnn.readNetFromCaffe(model_cfg, model_weights)

In [4]:
test_img = "macron.jpg"
image = cv2.imread(test_img)
(h, w) = image.shape[:2]

cv2.imshow('Test image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
# set the intensity scaling factor; 1 in this case, i.e. original image intensities
scalefactor = 1.0
 
# set the new dimensions for image resizing to match the network requirements
new_size = (300, 300)
 
# create a blob using OpenCV's DNN functionality and by performing mean subtraction 
# to normalize the input
blob = cv2.dnn.blobFromImage(image, scalefactor, new_size, (127.5, 127.5, 127.5), swapRB=True, crop=False)

# set the blob as input to the network
detector.setInput(blob)
# compute the forward pass - detect faces if any
detections = detector.forward()
detections.shape

(1, 1, 200, 7)

In [6]:
detections[0][0][0]
 
# compute the length of the array
len(detections[0][0])

# set the confidence threshold
confidence_threshold = 0.5

# loop over the detections
for i in range(0, detections.shape[2]):
  # extract the confidence (i.e., probability) associated with the prediction
  confidence = detections[0, 0, i, 2]
  # ignore weak detections
  if confidence > confidence_threshold:
    # compute the (x, y)-coordinates of the bounding box for the detected object
    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")
    # draw the bounding box of the detected face
    cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 255), 2)
    # print the probability of this detection
    text = "confidence: {:.2f}%".format(confidence * 100)
    y = startY - 10 if startY - 10 > 10 else startY + 10
    cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

# show the output image
cv2.imshow('Predicated Result',image)
cv2.waitKey(0)
cv2.destroyAllWindows()